# Configuração e Inicialização

In [1]:
!pip install pyspark # Instala o PySpark no ambiente Colab

In [2]:
import pyspark # Importa a biblioteca PySpark
from pyspark.sql import SparkSession # Importa a classe SparkSession, o ponto de entrada para DataFrames
from pyspark.sql.functions import col, upper # Importa funções SQL úteis. col é usada para referenciar colunas em operações, e upper para converter strings para maiúsculas

In [3]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém uma instância do SparkSession, inicializando o ambiente Spark

In [4]:
from google.colab import drive # Importa o módulo drive específico do Google Colab
drive.mount('/content/drive') # Monta o Google Drive no ambiente Colab, permitindo o acesso aos arquivos armazenados

Mounted at /content/drive


In [5]:
vendedores = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/vendedores.csv", header=True, inferSchema=True) # Lê o arquivo CSV vendedores.csv do Google Drive, criando um DataFrame. Usa a primeira linha como cabeçalho e infere os tipos de dados
itens_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True) # Lê o arquivo CSV itens_pedido.csv do Google Drive, criando um segundo DataFrame

# Transformações Básicas e Tipagem (Casting)

In [6]:
vendedores.show() # Exibe as primeiras linhas do DataFrame vendedores
vendedores.printSchema() # Exibe o esquema do DataFrame (nomes e tipos de dados das colunas) antes das transformações

+--------------------+------------+-----------------+---------------+
|         id_vendedor|cep_vendedor|  cidade_vendedor|estado_vendedor|
+--------------------+------------+-----------------+---------------+
|3442f8959a84dea7e...|       13023|         campinas|             SP|
|d1b65fc7debc3361e...|       13844|       mogi guacu|             SP|
|ce3ad9de960102d06...|       20031|   rio de janeiro|             RJ|
|c0f3eea2e14555b6f...|        4195|        sao paulo|             SP|
|51a04a8a6bdcb23de...|       12914|braganca paulista|             SP|
|c240c4061717ac180...|       20920|   rio de janeiro|             RJ|
|e49c26c3edfa46d22...|       55325|           brejao|             PE|
|1b938a7ec6ac5061a...|       16304|        penapolis|             SP|
|768a86e36ad6aae3d...|        1529|        sao paulo|             SP|
|ccc4bbb5f32a6ab2b...|       80310|         curitiba|             PR|
|8cb7c5ddf41f4d506...|       75110|         anapolis|             GO|
|a7a9b880c49781da6..

In [7]:
vendedores_tratados_df = vendedores.withColumn('cep_vendedor', vendedores.cep_vendedor.cast('string')) # Cria um novo DataFrame (vendedores_tratados_df) e converte o tipo de dado da coluna cep_vendedor para string

vendedores_tratados_df = vendedores_tratados_df.withColumn('cidade_vendedor_tratado', upper(vendedores['cidade_vendedor'])) # Adiciona uma nova coluna chamada cidade_vendedor_tratado que contém o conteúdo da coluna cidade_vendedor convertido para letras maiúsculas usando a função upper

vendedores_tratados_df.show() # Exibe o DataFrame transformado, mostrando as alterações nas colunas.
vendedores_tratados_df.printSchema() # Exibe o esquema do DataFrame transformado, confirmando os novos tipos ou colunas

+--------------------+------------+-----------------+---------------+-----------------------+
|         id_vendedor|cep_vendedor|  cidade_vendedor|estado_vendedor|cidade_vendedor_tratado|
+--------------------+------------+-----------------+---------------+-----------------------+
|3442f8959a84dea7e...|       13023|         campinas|             SP|               CAMPINAS|
|d1b65fc7debc3361e...|       13844|       mogi guacu|             SP|             MOGI GUACU|
|ce3ad9de960102d06...|       20031|   rio de janeiro|             RJ|         RIO DE JANEIRO|
|c0f3eea2e14555b6f...|        4195|        sao paulo|             SP|              SAO PAULO|
|51a04a8a6bdcb23de...|       12914|braganca paulista|             SP|      BRAGANCA PAULISTA|
|c240c4061717ac180...|       20920|   rio de janeiro|             RJ|         RIO DE JANEIRO|
|e49c26c3edfa46d22...|       55325|           brejao|             PE|                 BREJAO|
|1b938a7ec6ac5061a...|       16304|        penapolis|       

# Transformações em itens_pedido (Aritmética e Renomeação)

In [8]:
itens_pedido.show() # Exibe as primeiras linhas do DataFrame itens_pedido
itens_pedido.printSchema() # Exibe o esquema do DataFrame antes das transformações

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|valor_frete|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|      13.29|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|      19.93|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|      17.87|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|      12.79|
|00042b26cf59d7ce6...|             1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|      18.14|
|00048cc3ae777c65d...|             1|ef92defde84

In [13]:
# Cria um novo DataFrame aplicando uma sequência de transformações:

# Converte a coluna preco para o tipo numérico Float (ponto flutuante)
# Converte a coluna valor_frete para o tipo Float.
# Renomeia a coluna valor_frete para frete
# Adiciona uma nova coluna valor_total, que é o resultado da soma das colunas preco e frete.
itens_pedido_tratados_df = itens_pedido \
    .withColumn('preco', col('preco').cast('float')) \
    .withColumn('valor_frete', col('valor_frete').cast('float')) \
    .withColumnRenamed('valor_frete', 'frete') \
    .withColumn('valor_total', col('preco') + col('frete'))

itens_pedido_tratados_df.show() # Exibe as primeiras linhas do DataFrame transformado, mostrando a nova coluna valor_total

itens_pedido_tratados_df.printSchema() # Exibe o esquema, confirmando que preco, frete e valor_total são do tipo float

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|frete|valor_total|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|13.29|      72.19|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|19.93|     259.83|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|17.87|     216.87|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|12.79|  25.779999|
|00042b26cf59d7ce6...|             1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|18.14|     218.04|


# Manipulação de Data e Hora (to_date, date_format)

In [14]:
from pyspark.sql.functions import to_date, date_format # Importa funções SQL específicas para manipulação de data/hora.

itens_pedido_data_df = itens_pedido_tratados_df.withColumn("data", to_date(col("data_limite_envio"))) # Cria uma nova coluna chamada data convertendo a coluna data_limite_envio (que é Timestamp ou String) para o tipo Date (apenas data)

itens_pedido_data_df = itens_pedido_data_df.withColumn("data_br", date_format(col("data_limite_envio"), 'dd/MM/yyyy')) # Cria a coluna data_br formatando a data no formato brasileiro (dia/mês/ano) como uma string
itens_pedido_data_df = itens_pedido_data_df.withColumn("hora", date_format(col("data_limite_envio"), 'HH:mm:ss')) # Cria a coluna hora extraindo e formatando a parte da hora no formato HH:mm:ss como uma string

itens_pedido_data_df.show() # Exibe o DataFrame com as novas colunas de data e hora
itens_pedido_data_df.printSchema() # Exibe o esquema, mostrando que data é DateType e data_br/hora são StringType

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----+-----------+----------+----------+--------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|frete|valor_total|      data|   data_br|    hora|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----+-----------+----------+----------+--------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|13.29|      72.19|2017-09-19|19/09/2017|09:45:35|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|19.93|     259.83|2017-05-03|03/05/2017|11:05:13|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|17.87|     216.87|2018-01-18|18/01/2018|14:48:30|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d

# Conversão de Timestamp (Unix Time)

In [15]:
from pyspark.sql.functions import unix_timestamp, from_unixtime # Importa funções SQL para conversão de e para Unix Time (timestamp em segundos)

itens_pedido_data_reverso_df = itens_pedido_data_df.withColumn('timestamp', unix_timestamp(col('data_br'), 'dd/MM/yyyy')) # Converte a coluna string data_br (no formato dd/MM/yyyy) em um timestamp Unix (número de segundos desde 1970)

itens_pedido_data_reverso_df = itens_pedido_data_reverso_df.withColumn('data_formatada', from_unixtime('timestamp', 'yyyy/MM/dd')).withColumn('data_formatada', col('data_formatada').cast('date')) # (from_unixtime e cast): Primeiro, converte o timestamp de volta para uma string de data formatada (yyyy/MM/dd). Em seguida, converte essa string de data formatada para o tipo Date

itens_pedido_data_reverso_df.show() # Exibe o DataFrame, mostrando o timestamp numérico e a data final reformatada
itens_pedido_data_reverso_df.printSchema() # Exibe o esquema, mostrando que data_formatada é do tipo DateType

+--------------------+--------------+--------------------+--------------------+-------------------+------+-----+-----------+----------+----------+--------+----------+--------------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio| preco|frete|valor_total|      data|   data_br|    hora| timestamp|data_formatada|
+--------------------+--------------+--------------------+--------------------+-------------------+------+-----+-----------+----------+----------+--------+----------+--------------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|13.29|      72.19|2017-09-19|19/09/2017|09:45:35|1505779200|          NULL|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|19.93|     259.83|2017-05-03|03/05/2017|11:05:13|1493769600|          NULL|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 

# Escrita e Encerramento

In [16]:
itens_pedido_data_df.write.mode('overwrite').option('header', 'true').csv('output/itens_pedido_tratado_csv') # Salva o DataFrame: Escreve o DataFrame itens_pedido_data_df no formato CSV no diretório output/itens_pedido_tratado_csv. Usa o modo overwrite e inclui o cabeçalho

In [17]:
spark.read.option('header', 'true').csv('output/itens_pedido_tratado_csv').show() # Lê o arquivo CSV recém-salvo de volta no Spark e exibe o resultado para confirmar a escrita.

+--------------------+--------------+--------------------+--------------------+--------------------+------+-----+-----------+----------+----------+--------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|   data_limite_envio| preco|frete|valor_total|      data|   data_br|    hora|
+--------------------+--------------+--------------------+--------------------+--------------------+------+-----+-----------+----------+----------+--------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19T09:45:...|  58.9|13.29|      72.19|2017-09-19|19/09/2017|09:45:35|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03T11:05:...| 239.9|19.93|     259.83|2017-05-03|03/05/2017|11:05:13|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18T14:48:...| 199.0|17.87|     216.87|2018-01-18|18/01/2018|14:48:30|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|

In [18]:
spark.stop() # Encerra o SparkSession e libera os recursos alocados